In [1]:
import sys
import time
import json
import os
sys.path.insert(0, 'src/data')
sys.path.insert(0, 'src/dgl_graphsage')

import recommend as r
from train_updated import train
from api.spotifyAPI import SpotifyAPI

from sklearn.neighbors import NearestNeighbors
import pandas as pd

import matplotlib.pyplot as plt

Using backend: pytorch


In [2]:
%%time
# Loading Graph ~ 3min
feat_dir = "../../data/a13group1/features/merged_features.csv"
double_edge_dir = "../../data/a13group1/double_edges_170k.bin"
feat_data, adj_list, dgl_G, uri_map = r.load_double_edge(feat_dir, double_edge_dir)

Loading feature data...
Loaded DGL Graph
adj list created
CPU times: user 4min 6s, sys: 24.6 s, total: 4min 31s
Wall time: 4min 31s


In [3]:
%%time
# Training the Model. GPU ~ 00:00:40
with open('../../config/model-params.json') as fh:
            model_cfg = json.load(fh)
model, pred = train(dgl_G, feat_data, adj_list, **model_cfg)

# Put everything on CPU
model = model.to('cpu')
pred = pred.to('cpu')

Train pos edge: 26413580
Validation pos edge: 518464
Cuda enabled: True

Training starts:
In epoch 1 batch 1, loss: 0.611471951007843
In epoch 1 batch 6, loss: 0.502029299736023
In epoch 1 batch 11, loss: 0.3098103702068329
In epoch 1 batch 16, loss: 0.12996090948581696
In epoch 1 batch 21, loss: 0.1140880212187767

Epoch 1 AUC:  0.4110690675401437
In epoch 2 batch 1, loss: 0.1353190690279007
In epoch 2 batch 6, loss: 0.13941128551959991
In epoch 2 batch 11, loss: 0.1258445531129837
In epoch 2 batch 16, loss: 0.10559317469596863
In epoch 2 batch 21, loss: 0.10683856159448624

Epoch 2 AUC:  0.4323803597582683
In epoch 3 batch 1, loss: 0.1090640053153038
In epoch 3 batch 6, loss: 0.11176826804876328
In epoch 3 batch 11, loss: 0.10881184041500092
In epoch 3 batch 16, loss: 0.10278636962175369
In epoch 3 batch 21, loss: 0.10576053708791733

Epoch 3 AUC:  0.4561122345903865
In epoch 4 batch 1, loss: 0.10427933186292648
In epoch 4 batch 6, loss: 0.11137913912534714
In epoch 4 batch 11, loss:

In [4]:
# Create Embeddings
z = model(dgl_G, feat_data)

# Create Predictions
# In the form of the strength of the connection between source, destination from dgl_G.edges()
preds = pred(dgl_G, z)

In [8]:
# Create Nearest Neighbors
neigh = NearestNeighbors(n_neighbors=25, radius=0.4)
neigh.fit(feat_data)

NearestNeighbors(n_neighbors=25, radius=0.4)

In [9]:
# Get a random playlist
item = r.get_random_playlist()

/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.7000-7999.json


In [10]:
# Get playlist information
item, original_tracks, seeds = r.get_playlist_info(item)
original_tracks

Playlist ID: 7009
Playlist Length: 20


['Sneakin’---Drake',
 'Gassed Up---Nebu Kiniza',
 'You Was Right---Lil Uzi Vert',
 'Fire Squad - Live---J. Cole',
 'A Tale of 2 Citiez - Live---J. Cole',
 'Deja Vu---J. Cole',
 '20 Joints---Berner',
 'Moves---Big Sean',
 'Look At Me!---XXXTENTACION',
 'Rent Money---Future',
 'No Heart---21 Savage',
 'Dear Mama---2Pac',
 'Neighbors---J. Cole',
 '4 Your Eyez Only---J. Cole',
 'Miss America---J. Cole',
 'Like Toy Soldiers---Eminem',
 'T-Shirt---Migos',
 'Get Your Walk On---Xzibit',
 'X---Xzibit',
 'I Need A Doctor---Dr. Dre']

In [11]:
# Get recommendations
uri_recs = r.recommend(seeds, dgl_G, z, pred, neigh, feat_data, uri_map)
uri_recs

['47jNy3vE0aBM8nWWFAZaX2',
 '47jNy3vE0aBM8nWWFAZaX2',
 '5XlpERwWF8SuVUL9AghEIh',
 '6jalstnZd8hEJuLGFFjL9L',
 '5rSH2YlI18D4mmgP0tQbs9',
 '7dEYcnW1YSBpiKofefCFCf',
 '1eLU27PPFxBhbp1CBjPh9J',
 '3hxsFtkwEx6rZ310ROjwPD',
 '08azaGQ1YKWvajMDARPSeZ',
 '7dEYcnW1YSBpiKofefCFCf',
 '63Zqn50wQCs8G1fci05C8a',
 '0pSaDwrMLM7F82LWJg9v6s',
 '20WsdoJsGsjVOLtA0GB5hL',
 '5XlpERwWF8SuVUL9AghEIh',
 '7bFElqzAlHXnnINRBXRQbK',
 '7mcPcohfR96oIGbv6dxS3Q',
 '5XlpERwWF8SuVUL9AghEIh',
 '71u5SjnkSQgzIt1UzHchbi',
 '7crrp07AQujRvbOoIGitk8',
 '6B0FIZm6Xs4qYRyTjsuU0Y']

In [12]:
# Translate to Song Names
client_id = ''
client_secret = ''

spotify = SpotifyAPI(client_id, client_secret)

In [13]:
%%time
rec_track_names = r.get_rec_names(uri_recs, spotify, 2)

Exception: Could not authenticate client.

In [14]:
original_tracks

['Sneakin’---Drake',
 'Gassed Up---Nebu Kiniza',
 'You Was Right---Lil Uzi Vert',
 'Fire Squad - Live---J. Cole',
 'A Tale of 2 Citiez - Live---J. Cole',
 'Deja Vu---J. Cole',
 '20 Joints---Berner',
 'Moves---Big Sean',
 'Look At Me!---XXXTENTACION',
 'Rent Money---Future',
 'No Heart---21 Savage',
 'Dear Mama---2Pac',
 'Neighbors---J. Cole',
 '4 Your Eyez Only---J. Cole',
 'Miss America---J. Cole',
 'Like Toy Soldiers---Eminem',
 'T-Shirt---Migos',
 'Get Your Walk On---Xzibit',
 'X---Xzibit',
 'I Need A Doctor---Dr. Dre']

In [18]:
pd.Series(rec_track_names).value_counts()

Got It (feat. Snoop Dogg)---Boys Noize                                                                           3
Bad Meets Evil---Eminem                                                                                          2
Ka-Bang! (feat. MF Doom)---CZARFACE                                                                              2
Just Life---Mark Battles                                                                                         1
Inertia---ARTY                                                                                                   1
Love Yourz - Live---J. Cole                                                                                      1
1Train (feat. Kendrick Lamar, Joey Bada$$, Yelawolf, Danny Brown, Action Bronson & Big K.R.I.T.)---A$AP Rocky    1
Rap Up 2016---Uncle Murda                                                                                        1
Do Your Time---Ludacris                                                         

Note: We kept repeats because a song that is recommended as the best more than once throughout the playlist should be ranked higher